In [1]:
pip install praw

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import praw

In [3]:
reddit = praw.Reddit(
    client_id= 'Sf695hto-aPmp_vJvG5VRA',
    client_secret='l7FWeWpdjkhUu9UMNI2BvnUeDXai0w',
    user_agent='MyStockBot/1.0 by Salt-Interest6663'
)

In [4]:
subreddit = reddit.subreddit('stocks')  # Change to 'investing' or other relevant subreddits
posts = []
for submission in subreddit.new(limit=500):  # Fetch 500 latest posts
    posts.append({
        'title': submission.title,
        'selftext': submission.selftext,
        'score': submission.score,
        'comments': submission.num_comments,
        'created_utc': submission.created_utc
    })

In [5]:
import pandas as pd

df = pd.DataFrame(posts)
df.to_csv('reddit_stocks.csv', index=False)

In [6]:
import re
import pandas as pd

# Load the dataset
df = pd.read_csv('reddit_stocks.csv')

# Combine title and selftext
df['text'] = df['title'].fillna('') + ' ' + df['selftext'].fillna('')

# Remove noise (URLs, special characters, etc.)
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Lowercase
    return text

df['clean_text'] = df['text'].apply(clean_text)

In [7]:
df.head()

,title,selftext,score,comments,created_utc,text,clean_text
0,"Google, Apple, and Microsoft are facing anti t...",At some point do these companies momentum of g...,0,7,1.732782e+09,"Google, Apple, and Microsoft are facing anti t...",google apple and microsoft are facing anti tru...
1,Intel stock a viable investment now that it ha...,What are our thoughts on Intel? It’s a company...,40,40,1.732770e+09,Intel stock a viable investment now that it ha...,intel stock a viable investment now that it ha...
2,Microsoft faces wide-ranging U.S. antitrust probe,The U.S. Federal Trade Commission has opened a...,123,51,1.732758e+09,Microsoft faces wide-ranging U.S. antitrust pr...,microsoft faces wideranging us antitrust probe...
3,EC Stock - Value opportunity,"Hey all,\n\nI think I'm gonna open a position ...",2,4,1.732756e+09,"EC Stock - Value opportunity Hey all,\n\nI thi...",ec stock value opportunity hey all\n\ni think...
4,"PCE rises to 2.3% annually, ""meeting expectati...",I'm surprised the market has been taking this ...,62,32,1.732736e+09,"PCE rises to 2.3% annually, ""meeting expectati...",pce rises to 23 annually meeting expectations ...


## Data Analysis

In [8]:
pip install Textblob


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
from textblob import TextBlob

def get_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    return sentiment

df['sentiment'] = df['clean_text'].apply(get_sentiment)

In [10]:
#Feature Extraction


#Sentiment polarity (df['sentiment'])
#Frequency of mentions of specific stocks

In [11]:
def count_mentions(text, tickers):
    return sum(text.count(ticker) for ticker in tickers)

tickers = ['AAPL', 'TSLA', 'AMZN']  # List of stock tickers
df['mention_count'] = df['clean_text'].apply(lambda x: count_mentions(x, tickers))

In [12]:
df

,title,selftext,score,comments,created_utc,text,clean_text,sentiment,mention_count
0,"Google, Apple, and Microsoft are facing anti t...",At some point do these companies momentum of g...,0,7,1.732782e+09,"Google, Apple, and Microsoft are facing anti t...",google apple and microsoft are facing anti tru...,-0.018519,0
1,Intel stock a viable investment now that it ha...,What are our thoughts on Intel? It’s a company...,40,40,1.732770e+09,Intel stock a viable investment now that it ha...,intel stock a viable investment now that it ha...,0.000000,0
2,Microsoft faces wide-ranging U.S. antitrust probe,The U.S. Federal Trade Commission has opened a...,123,51,1.732758e+09,Microsoft faces wide-ranging U.S. antitrust pr...,microsoft faces wideranging us antitrust probe...,0.019010,0
3,EC Stock - Value opportunity,"Hey all,\n\nI think I'm gonna open a position ...",2,4,1.732756e+09,"EC Stock - Value opportunity Hey all,\n\nI thi...",ec stock value opportunity hey all\n\ni think...,0.088889,0
4,"PCE rises to 2.3% annually, ""meeting expectati...",I'm surprised the market has been taking this ...,62,32,1.732736e+09,"PCE rises to 2.3% annually, ""meeting expectati...",pce rises to 23 annually meeting expectations ...,0.153125,0
...,...,...,...,...,...,...,...,...,...
495,Investing $140k,My wife has student loans of 200k. We just cam...,0,42,1.729351e+09,Investing $140k My wife has student loans of 2...,investing 140k my wife has student loans of 20...,0.187778,0
496,Where do you keep your money in between buying...,Assuming a brokerage account...\n\nYou bought ...,59,118,1.729350e+09,Where do you keep your money in between buying...,where do you keep your money in between buying...,0.303333,0
497,Is a gamestop scenario happening in China righ...,"JD.com (HKG: 9618, NASDAQ: JD), a Chinese e-co...",0,3,1.729345e+09,Is a gamestop scenario happening in China righ...,is a gamestop scenario happening in china righ...,0.097581,0
498,Are there any stocks you will never buy becaus...,"For moral, ethical, religions etc reasons, is ...",303,1095,1.729343e+09,Are there any stocks you will never buy becaus...,are there any stocks you will never buy becaus...,0.128571,0


## Model buliding

In [13]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
import yfinance as yf
# Fetch historical stock data
stock_data = yf.download("AAPL", start="2023-01-01", end="2023-12-31")

[*********************100%***********************]  1 of 1 completed


In [15]:
stock_data['price_change'] = stock_data['Close'].pct_change()  # Percentage change
stock_data['movement'] = stock_data['price_change'].apply(lambda x: 1 if x > 0 else 0)

In [17]:
df['timestamp']

0     2024-11-28 08:23:41
1     2024-11-28 05:02:45
2     2024-11-28 01:38:14
3     2024-11-28 01:02:11
4     2024-11-27 19:35:42
              ...        
495   2024-10-19 15:12:28
496   2024-10-19 14:58:56
497   2024-10-19 13:36:35
498   2024-10-19 13:10:17
499   2024-10-19 11:01:23
Name: timestamp, Length: 500, dtype: datetime64[ns]

In [22]:
df['timestamp'] = pd.to_datetime(df['created_utc'], unit='s')  # Convert Reddit UTC
stock_data['timestamp'] = stock_data.index  # Assume stock data index is datetime

# Merge on timestamp (or within a time range)
merged_data = pd.merge_asof(df.sort_values('timestamp'), stock_data.sort_values('timestamp'), on='timestamp')
# Target variable

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [17]:
merged_data

,title,selftext,score,comments,created_utc,text,clean_text,sentiment,mention_count,timestamp,"(Adj Close, AAPL)","(Close, AAPL)","(High, AAPL)","(Low, AAPL)","(Open, AAPL)","(Volume, AAPL)","(price_change, )","(movement, )"
0,r/Stocks Daily Discussion & Technicals Tuesday...,"This is the daily discussion, so anything stoc...",9,280,1.728985e+09,r/Stocks Daily Discussion & Technicals Tuesday...,rstocks daily discussion technicals tuesday ...,0.008681,0,2024-10-15 09:30:11,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
1,Management fees,"Hello folks, \nI was just a little curious how...",8,6,1.728995e+09,"Management fees Hello folks, \nI was just a li...",management fees hello folks \ni was just a lit...,0.105357,0,2024-10-15 12:15:57,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
2,Google to buy nuclear power for AI datacentres...,https://www.theguardian.com/technology/2024/oc...,526,76,1.728999e+09,Google to buy nuclear power for AI datacentres...,google to buy nuclear power for ai datacentres...,0.096591,0,2024-10-15 13:26:45,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
3,Boeing to Sell at Least $10 Billion in Shares ...,*source:* [*https://www.wsj.com/business/boein...,421,128,1.728999e+09,Boeing to Sell at Least $10 Billion in Shares ...,boeing to sell at least 10 billion in shares t...,0.026439,0,2024-10-15 13:30:42,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
4,Semis take a hit after news on limiting exports,The semi industry is taking a hit after news a...,125,44,1.729004e+09,Semis take a hit after news on limiting export...,semis take a hit after news on limiting export...,0.071875,0,2024-10-15 14:57:03,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Intuit shares drop as quarterly forecast misse...,Intuit shares fell 6% in extended trading Thur...,9,0,1.732244e+09,Intuit shares drop as quarterly forecast misse...,intuit shares drop as quarterly forecast misse...,0.116092,0,2024-11-22 02:58:33,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
496,"OK, Here is the main reason that why Biotech c...",My original post here [Why is NVO willing to s...,32,15,1.732246e+09,"OK, Here is the main reason that why Biotech c...",ok here is the main reason that why biotech co...,0.172917,0,2024-11-22 03:19:05,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
497,I Still Can’t Understand WHY? And Why Keep Goi...,u/[G-Money1965](https://www.reddit.com/user/G-...,0,11,1.732252e+09,I Still Can’t Understand WHY? And Why Keep Goi...,i still cant understand why and why keep going...,-0.036064,0,2024-11-22 05:02:30,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0
498,Why is selling a limit order after hours risky?,I’m scheduling an after hours sell for the fir...,3,4,1.732255e+09,Why is selling a limit order after hours risky...,why is selling a limit order after hours risky...,0.191667,0,2024-11-22 05:54:57,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0


In [18]:
threshold = 0.02  # Define a 2% movement threshold

def classify_movement(change):
    if change > threshold:
        return 1  # Upward movement
    elif change < -threshold:
        return -1  # Downward movement
    else:
        return 0  # Neutral

stock_data['movement'] = stock_data['price_change'].apply(classify_movement)

In [19]:
stock_data['next_day_movement'] = stock_data['movement'].shift(-1)

# Merge Reddit data with stock movement
merged_data = pd.merge_asof(df.sort_values('timestamp'), stock_data.sort_values('timestamp'), on='timestamp')


C:\Users\hp\AppData\Local\Temp\ipykernel_16140\1225964181.py:4: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  merged_data = pd.merge_asof(df.sort_values('timestamp'), stock_data.sort_values('timestamp'), on='timestamp')


In [21]:
merged_data

,title,selftext,score,comments,created_utc,text,clean_text,sentiment,mention_count,timestamp,"(Adj Close, AAPL)","(Close, AAPL)","(High, AAPL)","(Low, AAPL)","(Open, AAPL)","(Volume, AAPL)","(price_change, )","(movement, )","(next_day_movement, )"
0,r/Stocks Daily Discussion & Technicals Tuesday...,"This is the daily discussion, so anything stoc...",9,280,1.728985e+09,r/Stocks Daily Discussion & Technicals Tuesday...,rstocks daily discussion technicals tuesday ...,0.008681,0,2024-10-15 09:30:11,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
1,Management fees,"Hello folks, \nI was just a little curious how...",8,6,1.728995e+09,"Management fees Hello folks, \nI was just a li...",management fees hello folks \ni was just a lit...,0.105357,0,2024-10-15 12:15:57,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
2,Google to buy nuclear power for AI datacentres...,https://www.theguardian.com/technology/2024/oc...,526,76,1.728999e+09,Google to buy nuclear power for AI datacentres...,google to buy nuclear power for ai datacentres...,0.096591,0,2024-10-15 13:26:45,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
3,Boeing to Sell at Least $10 Billion in Shares ...,*source:* [*https://www.wsj.com/business/boein...,421,128,1.728999e+09,Boeing to Sell at Least $10 Billion in Shares ...,boeing to sell at least 10 billion in shares t...,0.026439,0,2024-10-15 13:30:42,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
4,Semis take a hit after news on limiting exports,The semi industry is taking a hit after news a...,125,44,1.729004e+09,Semis take a hit after news on limiting export...,semis take a hit after news on limiting export...,0.071875,0,2024-10-15 14:57:03,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Intuit shares drop as quarterly forecast misse...,Intuit shares fell 6% in extended trading Thur...,9,0,1.732244e+09,Intuit shares drop as quarterly forecast misse...,intuit shares drop as quarterly forecast misse...,0.116092,0,2024-11-22 02:58:33,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
496,"OK, Here is the main reason that why Biotech c...",My original post here [Why is NVO willing to s...,32,15,1.732246e+09,"OK, Here is the main reason that why Biotech c...",ok here is the main reason that why biotech co...,0.172917,0,2024-11-22 03:19:05,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
497,I Still Can’t Understand WHY? And Why Keep Goi...,u/[G-Money1965](https://www.reddit.com/user/G-...,0,11,1.732252e+09,I Still Can’t Understand WHY? And Why Keep Goi...,i still cant understand why and why keep going...,-0.036064,0,2024-11-22 05:02:30,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN
498,Why is selling a limit order after hours risky?,I’m scheduling an after hours sell for the fir...,3,4,1.732255e+09,Why is selling a limit order after hours risky...,why is selling a limit order after hours risky...,0.191667,0,2024-11-22 05:54:57,191.591385,192.529999,194.399994,191.729996,193.899994,42628800,-0.005424,0,NaN


In [25]:
X

(                                                 title  \
 0    r/Stocks Daily Discussion & Technicals Tuesday...   
 1                                      Management fees   
 2    Google to buy nuclear power for AI datacentres...   
 3    Boeing to Sell at Least $10 Billion in Shares ...   
 4     Semis take a hit after news on limiting exports    
 ..                                                 ...   
 495  Intuit shares drop as quarterly forecast misse...   
 496  OK, Here is the main reason that why Biotech c...   
 497  I Still Can’t Understand WHY? And Why Keep Goi...   
 498    Why is selling a limit order after hours risky?   
 499  If a new product is brought to market, is it's...   
 
                                               selftext  score  comments  \
 0    This is the daily discussion, so anything stoc...      9       280   
 1    Hello folks, \nI was just a little curious how...      8         6   
 2    https://www.theguardian.com/technology/2024/oc...    526

In [26]:
y

0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: (movement, ), Length: 500, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split

X = merged_data.iloc[:, :-3],  # Example features

y = merged_data.iloc[:, -2]  
  # Load or calculate stock movements


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)